# Devoir

## Setup

In [1]:
# Libraries

## System
import os
import shutil
import yaml

# Create data directories
for folder in ["Data", os.path.join("Data","Raw"), os.path.join("Data","Modified"), os.path.join("Data","Final"), os.path.join("API_key")]:
    if not os.path.isdir(folder):
        os.makedirs(folder)

## Data wrangling
import pandas as pd
import numpy as np
import feather  # pip install feather-format

## DL/Extracting
import gzip
import urllib.request
import requests
from bs4 import BeautifulSoup

## Stats
from scipy import stats

## Custom functions
import sks_functions as sks


#########################################################################################################
# OMBd API key
if not os.path.exists(os.path.join("API_key", "key.yaml")):
    print("Please put an OMDb API key in a file called key.yaml. Or don't, if you don't want to bother with getting data from OMDb.")
else:
    with open(os.path.join("API_key", "key.yaml"), 'r') as stream:
        OMDb_key = yaml.safe_load(stream)

#########################################################################################################
# OMDb call function
# * Don't forget to put your API key in API_Key/key.yaml

def getOMDbMovieData(_df, filename_csv="df_omdb.csv"):
    api_call = requests.get("http://www.omdbapi.com/?apikey="+OMDb_key+"&i="+_df["tconst"]+"&plot=full")
    if api_call.json()["Response"]=="True":
        # Some movies are wrongly (?) labelled "episode" and "game" on OMDb
        # I'll trust IMDb "titleType" variable
        assert api_call.json()["Type"] in ['movie', 'series', 'episode', 'game'], "Type not movie, series or episode"
        if api_call.json()["Type"]=="movie":
            _df["api_title"]=api_call.json()["Title"]
            _df["rated"]=api_call.json()["Rated"]
            _df["awards"]=api_call.json()["Awards"]
            _df["country"]=api_call.json()["Country"]
            _df["language"]=api_call.json()["Language"]
            _df["plot"]=api_call.json()["Plot"]
            _df["boxoffice"]=api_call.json()["BoxOffice"]
            _df["production"]=api_call.json()["Production"]
        else:
            _df["api_title"]=api_call.json()["Title"]
            _df["rated"]=api_call.json()["Rated"]
            _df["awards"]=api_call.json()["Awards"]
            _df["country"]=api_call.json()["Country"]
            _df["language"]=api_call.json()["Language"]
            _df["plot"]=api_call.json()["Plot"]
            _df["boxoffice"]=np.NaN
            _df["production"]=np.NaN
    else:
        _df["api_title"]=np.NaN
        _df["rated"]=np.NaN
        _df["awards"]=np.NaN
        _df["country"]=np.NaN
        _df["language"]=np.NaN
        _df["plot"]=np.NaN
        _df["boxoffice"]=np.NaN
        _df["production"]=np.NaN
    
    #Write to csv, for backup
    if not os.path.isfile(os.path.join("Data", "Modified", filename_csv)):
        pd.DataFrame(_df).transpose().to_csv(os.path.join("Data", "Modified", filename_csv), index=False)
    else:
        pd.DataFrame(_df).transpose().to_csv(os.path.join("Data", "Modified", filename_csv), mode='a', header=False, index=False)
    return _df        

**NB**: data is updated daily.

In [3]:
# Downloading dfs
for file_name in ["name.basics.tsv", "title.akas.tsv", "title.basics.tsv", 
"title.crew.tsv", "title.episode.tsv", "title.principals.tsv", "title.ratings.tsv"]:
    url_df = "https://datasets.imdbws.com/"
    comp_format = ".gz"
    compressed_df = os.path.join("Data","Raw",file_name+comp_format)
    uncompressed_df = os.path.join("Data","Raw",file_name)

    if not os.path.isfile(uncompressed_df):
        print("Downloading", file_name+comp_format)
        with urllib.request.urlopen(url_df+file_name+comp_format) as request, open(compressed_df, 'wb') as output:
            shutil.copyfileobj(request, output)
        with gzip.open(compressed_df, 'rb') as i:
            with open(uncompressed_df, 'wb') as o:
                shutil.copyfileobj(i, o)
        if os.path.isfile(compressed_df):
            os.remove(compressed_df)
    else:
        print("Skipping",file_name,"(already exists)")

Skipping name.basics.tsv (already exists)
Skipping title.akas.tsv (already exists)
Skipping title.basics.tsv (already exists)
Skipping title.crew.tsv (already exists)
Skipping title.episode.tsv (already exists)
Skipping title.principals.tsv (already exists)
Skipping title.ratings.tsv (already exists)


---
## Data Wrangling
---

In [ ]:
# Import all csv

# for df in ["name.basics.tsv", "title.akas.tsv", "title.basics.tsv", 
# "title.crew.tsv", "title.episode.tsv", "title.principals.tsv", "title.ratings.tsv"]:
#     path = os.path.join("Data","Raw",df)
#     df = pd.read_table(path)

---
### name.basics
---

Used variables: 
* sumVotes: sum of votes of the 4 main films a person has participated in
* avgRating: (weighted) average rating of the 4 main films a person has participated in

        name.basics.tsv.gz – Contains the following information for names:

                nconst (string) - alphanumeric unique identifier of the name/person -> OK
                primaryName (string)– name by which the person is most often credited -> OK
                birthYear – in YYYY format -> Rejet
                deathYear – in YYYY format if applicable, else '\N' -> Rejet
                primaryProfession (array of strings)– the top-3 professions of the person -> OK?
                knownForTitles (array of tconsts) – titles the person is known for -> OK?

In [5]:
name_basics = (
    pd.read_table(
        os.path.join("Data", "Raw", "name.basics.tsv"),
        na_values="\\N",
        usecols=["nconst", "primaryName", "primaryProfession", "knownForTitles"],
    )
    .loc[lambda x: x["primaryProfession"].notna() | x["knownForTitles"].notna(), :]
    .assign(
        **{
            "Profession1": (
                lambda x: x["primaryProfession"].str.split(",", expand=True)[0]
            ),
            "n_knownForTitles": (
                lambda x: x["knownForTitles"].str.split(",").str.len().fillna(0)
            ),
        }
    )
)

#### Creating avgRating and sumVotes

In [6]:
##########################################################################
# Merge du dataset name.basics avec les notes des films dans lesquels ils ont joué
# Long. Bien ~200s sur mon PC.
##########################################################################

# Listes dans var "knownForTitles" vers nouvelles colonnes
# NB: quelques valeurs sont des listes de 5 ou 6 films, je les enlève pour harmoniser (et éviter deux colonnes en plus)
name_basics_tmp = pd.concat(
        [name_basics.loc[:,["nconst", "primaryName", "knownForTitles"]],
        name_basics["knownForTitles"].str.split(",", expand=True).loc[:,0:3].rename(columns={0:"Film1", 1:"Film2", 2:"Film3", 3:"Film4"})],
        axis=1
    )

# Pivot "Film1":"Film4" to rows (one per film)
# + delete NaN rows (if known for less than 4 films)
name_basics_tmp = pd.wide_to_long(
    df=name_basics_tmp, stubnames="Film", i="nconst", j="rank_film").reset_index().rename(columns={"Film":"tconst"}
    ).loc[lambda x: x["tconst"].notna(),:]

# Merge name_basics avec note et nb de votes de chaque film (title.ratings)
# NB: certains films n'ont pas de note
name_basics_tmp = pd.merge(
    name_basics_tmp,
    pd.read_table(os.path.join("Data", "Raw", "title.ratings.tsv"), na_values="\\N"),
    how="left",
    on=["tconst"]
)

# Format long -> wide
# Une colonne par id_film (1 à 4), rating (1 à 4), vote (1 à 4)
name_basics_tmp = pd.pivot(name_basics_tmp, index=["nconst", "primaryName", "knownForTitles"], columns=["rank_film"]).reset_index()

# Suppression des index, et multiIndexing colonnes
name_basics_tmp.columns = name_basics_tmp.columns.to_flat_index()
name_basics_tmp = name_basics_tmp.rename(columns={
    ('averageRating', 1): "averageRating1",
    ('averageRating', 2): "averageRating2",
    ('averageRating', 3): "averageRating3",
    ('averageRating', 4): "averageRating4",
    ('knownForTitles', ''): "knownForTitles",
    ('nconst', ''): "nconst",
    ('numVotes', 1): "numVotes1",
    ('numVotes', 2): "numVotes2",
    ('numVotes', 3): "numVotes3",
    ('numVotes', 4): "numVotes4",
    ('primaryName', ''): "primaryName",
    ('tconst', 1): "tconst1",
    ('tconst', 2): "tconst2",
    ('tconst', 3): "tconst3",
    ('tconst', 4): "tconst4"
})

In [7]:
##################################
# Checks
##################################

# OK
# À noter que pas mal de films n'ont pas de note
name_basics_tmp.sample(5).head()

,nconst,primaryName,knownForTitles,tconst1,tconst2,tconst3,tconst4,averageRating1,averageRating2,averageRating3,averageRating4,numVotes1,numVotes2,numVotes3,numVotes4
6585855,nm6389784,Jelena Milenkovic,tt3629372,tt3629372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1717680,nm11592218,Samuel Barros,tt8811948,tt8811948,NaN,NaN,NaN,8.4,NaN,NaN,NaN,11.0,NaN,NaN,NaN
2495620,nm1385240,Kenneth Gracey,"tt1158939,tt0385961,tt0285685,tt0347411",tt1158939,tt0385961,tt0285685,tt0347411,5.9,3.5,3.9,3.4,3306.0,467.0,1080.0,258.0
225565,nm0239543,Dawn Dubovsky,tt0114202,tt0114202,NaN,NaN,NaN,4.2,NaN,NaN,NaN,991.0,NaN,NaN,NaN
8657425,nm9310521,Claire Soister,tt7252510,tt7252510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
## Getting rid of actors who have played in films with no ratings
name_basics_tmp = name_basics_tmp.drop(
    [el for el in name_basics_tmp.loc[lambda x: x["averageRating1"].isna() & x["averageRating2"].isna() & x["averageRating3"].isna() & x["averageRating4"].isna(),:].index]
)

## New vars of interest: 
## Number of votes across all films in which the obs. has been involved
## Avg rating films in which the obs. has been involved
name_basics_tmp = name_basics_tmp.assign(
    sumVotes = lambda x: x["numVotes1"].fillna(0) + x["numVotes2"].fillna(0) + x["numVotes3"].fillna(0) + x["numVotes4"].fillna(0),
    avgRating = lambda x: (x["numVotes1"].fillna(0)  * x["averageRating1"].fillna(0) + x["numVotes2"].fillna(0)  * x["averageRating2"].fillna(0) + x["numVotes3"].fillna(0)  * x["averageRating3"].fillna(0) + x["numVotes4"].fillna(0) * x["averageRating4"].fillna(0) ) / x["sumVotes"]
)

name_basics_tmp.head()

,nconst,primaryName,knownForTitles,tconst1,tconst2,tconst3,tconst4,averageRating1,averageRating2,averageRating3,averageRating4,numVotes1,numVotes2,numVotes3,numVotes4,sumVotes,avgRating
0,nm0000001,Fred Astaire,"tt0053137,tt0031983,tt0072308,tt0050419",tt0053137,tt0031983,tt0072308,tt0050419,7.2,6.9,7.0,7.0,12187.0,2357.0,40664.0,28016.0,83224.0,7.026455
1,nm0000002,Lauren Bacall,"tt0071877,tt0117057,tt0038355,tt0037382",tt0071877,tt0117057,tt0038355,tt0037382,7.3,6.6,7.9,7.8,57235.0,17222.0,79856.0,31775.0,186088.0,7.578071
2,nm0000003,Brigitte Bardot,"tt0049189,tt0057345,tt0056404,tt0054452",tt0049189,tt0057345,tt0056404,tt0054452,6.3,7.6,5.5,7.6,7810.0,29685.0,517.0,3308.0,41320.0,7.328008
3,nm0000004,John Belushi,"tt0072562,tt0078723,tt0080455,tt0077975",tt0072562,tt0078723,tt0080455,tt0077975,8.0,5.8,7.9,7.5,44280.0,31826.0,186907.0,113769.0,376782.0,7.613590
4,nm0000005,Ingmar Bergman,"tt0069467,tt0050976,tt0050986,tt0060827",tt0069467,tt0050976,tt0050986,tt0060827,8.1,8.2,8.2,8.1,31161.0,169745.0,99141.0,107092.0,407139.0,8.166043


In [9]:
# Vars of interest
name_basics_tmp = name_basics_tmp.loc[:,["nconst", "sumVotes", "avgRating"]]

In [11]:
#To feather, for merging with name_basics
name_basics_tmp.reset_index().drop(["index"], axis=1).to_feather(os.path.join("Data", "Modified", "name.film.ratings.feather"))

# Freeing memory
del name_basics
del name_basics_tmp

---
### title_akas
---

New used vars:
* Number of different titles (across geographical regions) for the film

        title.akas.tsv.gz - Contains the following information for titles:

            titleId (string) - a tconst, an alphanumeric unique identifier of the title
            ordering (integer) – a number to uniquely identify rows for a given titleId
            title (string) – the localized title
            region (string) - the region for this version of the title
            language (string) - the language of the title
            types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
            attributes (array) - Additional terms to describe this alternative title, not enumerated
            isOriginalTitle (boolean) – 0: not original title; 1: original title

Liste les différents noms des films, par région. On pourrait l'utiliser pour obtenir un indicateur de "l'exportation" du film : si le titre du film est traduit dans plusieurs langues, c'est qu'il est sorti dans plusieurs pays.
À noter qu'on aura des erreurs de mesure: les titres des films ne sont pas systématiquement traduits.

Hypothétiquement, si le titre film a été traduit dans d'autres langues, c'est qu'il a réussi à s'exporter. Il pourrait avoir un meilleur score que les autres.

Malheureusement il ne semble pas possible de pouvoir retrouver facilement de quel pays vient un film à partir de ce dataset.

In [13]:
title_akas = pd.read_table(os.path.join("Data","Raw","title.akas.tsv"),
                           na_values="\\N",
                           dtype= {'attributes': 'str',
                                   'isOriginalTitle': 'Int8',
                                   'language': 'category',
                                   'ordering': 'int16',
                                   'region': 'category',
                                   'title': 'str',
                                   'titleId': 'str',
                                   'types': 'category'})
title_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [14]:
title_akas = (pd.DataFrame(title_akas.groupby("titleId").count()["title"])
                .rename(columns={"title":"n_titles"})
)

In [15]:
title_akas.reset_index().to_feather(os.path.join("Data", "Modified", "ntitles.feather"))

del title_akas

---
### title.crew
---

No new vars. À importer dans df principal ?

    title.crew.tsv.gz – Contains the director and writer information for all the titles in IMDb. Fields include:

        tconst (string) - alphanumeric unique identifier of the title
        directors (array of nconsts) - director(s) of the given title
        writers (array of nconsts) – writer(s) of the given title

In [145]:
title_crew = pd.read_table(os.path.join("Data","Raw","title.crew.tsv"), na_values="\\N")

In [25]:
title_crew.sample(5).head()

,tconst,directors,writers
4752121,tt2887764,NaN,NaN
5170396,tt3861978,nm6630046,nm6630046
4754726,tt2893778,nm0008408,"nm1104059,nm0860809,nm2629633,nm0022935,nm0101..."
6426938,tt6670328,NaN,NaN
1650032,tt11192534,nm1815399,NaN


In [26]:
del title_crew

---
### title.episode
---

Rien d'utilisé dans le df. Potentiellement le nombre de saisons, si je travaille uniquement sur des séries.

    title.episode.tsv.gz – Contains the tv episode information. Fields include:

        tconst (string) - alphanumeric identifier of episode
        parentTconst (string) - alphanumeric identifier of the parent TV Series
        seasonNumber (integer) – season number the episode belongs to
        episodeNumber (integer) – episode number of the tconst in the TV series

In [91]:
title_episode = pd.read_table(os.path.join("Data", "Raw", "title.episode.tsv"), na_values="\\N")

In [92]:
title_episode.head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3.0,42.0
4,tt0043631,tt0989125,2.0,16.0


In [94]:
title_episode = title_episode.groupby("parentTconst").agg(maxSeason=("seasonNumber", "max"))
title_episode.reset_index().to_feather(os.path.join("Data", "Modified", "series.seasons.feather"))

In [95]:
del title_episode

---
### title.principals
---

    title.principals.tsv.gz – Contains the principal cast/crew for titles

        tconst (string) - alphanumeric unique identifier of the title -> OK
        ordering (integer) – a number to uniquely identify rows for a given titleId -> OK
        nconst (string) - alphanumeric unique identifier of the name/person -> OK
        category (string) - the category of job that person was in -> Perdu lors du pivot
        job (string) - the specific job title if applicable, else '\N' -> Perdu lors du pivot
        characters (string) - the name of the character played if applicable, else '\N' -> Perdu lors du pivot

'title_principals' contient des données sur les principaux principaux acteurs du film.
On s'attendra à ce que des acteurs ou réalisateurs revenant souvent (on utilisera le nombre de votes comme variable proxy) tendent à faire monter le rating d'un film, soit : 
* Par expérience, en jouant dans beaucoup de film
* Parce qu'ils sont connus du public

On pourra aussi utiliser le weighted average rating des films dans lesquels un acteur a joué comme proxy pour mesurer s'il est apprécié ou non du public (ou au moins s'il est associé à des "bons" films)

In [2]:
# Merge this table with the sum of votes & average rating of all the films
# a person has worked on

title_principals = pd.merge(
    pd.read_table(
        os.path.join("Data", "Raw", "title.principals.tsv"),
        usecols=["tconst", "ordering", "nconst", "category"],
        na_values="\\N",
    ),
    pd.read_feather(os.path.join("Data", "Modified", "name.film.ratings.feather")),
    how="left",
    on=["nconst"],
).assign(
    category=lambda x: x["category"].astype("category"),
    # Collapse "category" to fewer categories
    category_r=lambda x: x["category"].replace(
        sks.fct_collapse(
            {
                "Actors": ["actor", "actress", "self"],
                "Director": ["director"],
                "Others": [
                    "archive_footage",
                    "archive_sound",
                    "cinematographer",
                    "composer",
                    "editor",
                    "producer",
                    "production_designer",
                    "writer",
                ],
            }
        )
    ).astype("category")
)

# Crew size indice. On regardera si le nombre de lignes par film dans ce dataset a un quelconque lien avec les ratings
title_principals.groupby("tconst").agg(crew_size=("ordering", "max")).rename(
    columns={"crew_size": "crew_size_indice"}
).reset_index().to_feather(os.path.join("Data", "Modified", "crew.size.indice.ftr"))

# Drop NaN in the sumVotes & avgRating comlumns. Otherwise, NaN will screw up the weighted average calculations
title_principals = title_principals.dropna(axis=0, how="any", subset=["sumVotes", "avgRating"])

title_principals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39828278 entries, 0 to 45371571
Data columns (total 7 columns):
 #   Column      Dtype   
---  ------      -----   
 0   tconst      object  
 1   ordering    int64   
 2   nconst      object  
 3   category    category
 4   sumVotes    float64 
 5   avgRating   float64 
 6   category_r  category
dtypes: category(2), float64(2), int64(1), object(2)
memory usage: 1.9+ GB


In [9]:
# Get crew names

names = pd.merge(
    title_principals.loc[:,["tconst", "nconst", "category_r"]],
    pd.read_table(
        os.path.join("Data", "Raw", "name.basics.tsv"),
        na_values="\\N",
        usecols=["nconst", "primaryName"],
    ),
    how="left",
    on="nconst"
)

names = names.groupby(["tconst", "category_r"]).agg(name_=("primaryName", lambda x: ", ".join(x))).reset_index().pivot(index="tconst", columns="category_r")
names.columns = names.columns.to_flat_index()
names = names.rename(columns={
    ('name_', 'Actors'): "names_Actors",
    ('name_', 'Director'): "names_Director",
    ('name_', 'Others'): "names_Others",
})

names.reset_index().to_feather(os.path.join("Data", "Modified", "crew.names.feather"))

del names

In [16]:
title_principals = pd.merge(
    title_principals,
    pd.read_feather(
        os.path.join("Data", "Modified", "crew.names.feather")
    ),
    how="left",
    on="tconst"
)

In [19]:
# TEST TEST
# tmp = title_principals.sort_values("tconst").iloc[35000000:35001000]

In [26]:
#Approximation de la taille de l'équipe impliquée dans la réalisation? Ou plutôt du nombre de "stars"?
#Max de 10. Pas sûr de l'utilité.
title_principals["ordering"].describe()

count    3.982828e+07
mean     4.629349e+00
std      2.796736e+00
min      1.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      7.000000e+00
max      1.000000e+01
Name: ordering, dtype: float64

In [27]:
# Sanity check
pd.crosstab(title_principals["category"], title_principals["category_r"])

category_r,Actors,Director,Others
category,,,
actor,8986211,0,0
actress,6917760,0,0
archive_footage,0,0,245945
archive_sound,0,0,2654
cinematographer,0,0,1311175
composer,0,0,1408061
director,0,4412670,0
editor,0,0,1300496
producer,0,0,2455674


In [34]:
# This yields, for the director/actors/other staff:
# - the sum of all votes across all films they have participated in
# - the average rating across all films they have participated in 
#   *** Scratch that, I deleted it to save memory
# - the weighted (on the number of votes) average across all films they have participated in (not working currently, need to optimise)

#### C'est LONG. Une bonne heure, à cause de la fonction lambda. Je sais pas s'il y a un moyen plus efficace (sans paralléliser).

title_principals = title_principals.groupby(["tconst", "category_r"]).agg(
        sumVotes=("sumVotes", "sum"),
        #avgRating=("avgRating", "mean"),
        avgRating_weighted=("avgRating", lambda x: np.average(x, weights=title_principals.loc[x.index, "sumVotes"]))
        ).reset_index().pivot(index="tconst", columns="category_r")

title_principals.columns = title_principals.columns.to_flat_index()

title_principals = title_principals.rename(columns={
    ('sumVotes', 'Actors'): "sumVotes_Actors" ,
    ('sumVotes', 'Director'): "sumVotes_Director" ,
    ('sumVotes', 'Others'): "sumVotes_Others" ,
    # ('avgRating', 'Actors'): "avgRating_Actors" ,
    # ('avgRating', 'Director'): "avgRating_Director" ,
    # ('avgRating', 'Others'): "avgRating_Others",
    ('avgRating_weighted', 'Actors'): "avgRating_weighted_Actors",
    ('avgRating_weighted', 'Director'): "avgRating_weighted_Director",
    ('avgRating_weighted', 'Others'): "avgRating_weighted_Others" 
})

title_principals.sample(5).head()

,sumVotes_Actors,sumVotes_Director,sumVotes_Others,avgRating_weighted_Actors,avgRating_weighted_Director,avgRating_weighted_Others
tconst,,,,,,
tt9563800,107.0,0.0,0.0,6.374766,NaN,NaN
tt8861254,0.0,4358.0,2388.0,NaN,5.134052,7.779229
tt4481078,145017.0,206.0,154754.0,7.857952,8.145631,6.989271
tt5688492,3527997.0,0.0,0.0,6.832185,NaN,NaN
tt6265766,2526571.0,291.0,69452.0,7.984927,7.437113,6.900271


Some directors are missing from this dataset. They (and their votes/scores) can be retrieved from the title.crew dataset. 

In [47]:
# Get directors for each film
# We'll keep only the main one, otherwise I would have to pivot the table again.
title_principals = pd.merge(
    title_principals,
    pd.read_table(os.path.join("Data","Raw","title.crew.tsv"), na_values="\\N", usecols=["tconst", "directors"]),
    how="left",
    on="tconst"
).assign(
    directors = lambda x: x["directors"].str.split(",").str[0]
)

title_principals.sample(5).head()

,tconst,sumVotes_Actors,sumVotes_Director,sumVotes_Others,avgRating_weighted_Actors,avgRating_weighted_Director,avgRating_weighted_Others,directors
5389947,tt6077248,807417.0,89289.0,29785.0,7.549282,5.900207,7.275118,nm1308450
1891743,tt11922488,0.0,0.0,3492.0,NaN,NaN,7.557073,NaN
3306310,tt1507385,672.0,0.0,0.0,6.508929,NaN,NaN,NaN
5104738,tt5308712,45860.0,4978.0,51854.0,8.087933,5.442025,7.881029,nm0170913
6321398,tt8501798,98989.0,344.0,1626.0,6.615021,6.489826,6.512608,nm0194720


In [65]:
#Get the votes & ratings of each director
title_principals = pd.merge(
    title_principals.rename(columns={"directors": "nconst"}),
    pd.read_feather(os.path.join("Data", "Modified", "name.film.ratings.feather")),
    how="left",
    on="nconst"
)

#Replace missing values
## Mask: directors whose votes & ratings are NaN
title_principals["missing_loc"] = np.where(
    (title_principals["sumVotes_Director"]==0) &
    (title_principals["avgRating_weighted_Director"].isna()) &
    (title_principals["sumVotes"].notna()) &
    (title_principals["avgRating"].notna()),
    1, 0)
## Replacing NaN with data
title_principals.loc[title_principals["missing_loc"]==1, "sumVotes_Director"] = title_principals.loc[title_principals["missing_loc"]==1, "sumVotes"]
title_principals.loc[title_principals["missing_loc"]==1, "avgRating_weighted_Director"] = title_principals.loc[title_principals["missing_loc"]==1, "avgRating"]
title_principals = title_principals.drop(['nconst', 'sumVotes', 'avgRating', 'missing_loc'], axis=1)

title_principals.sample(5).head()

,tconst,sumVotes_Actors,sumVotes_Director,sumVotes_Others,avgRating_weighted_Actors,avgRating_weighted_Director,avgRating_weighted_Others
2025192,tt12192444,12369.0,453.0,5460.0,6.879287,7.400000,6.182509
4515521,tt3678208,86527.0,168836.0,7721.0,7.072372,6.439667,7.154293
4914036,tt4779002,63.0,0.0,0.0,4.603175,NaN,NaN
6353167,tt8583032,74836.0,73770.0,929527.0,7.957428,7.974023,7.487024
3953769,tt2269386,23.0,12.0,53.0,7.700000,7.100000,6.856604


In [66]:
title_principals.to_feather(os.path.join("Data","Modified","crew.ratings.feather"))

del title_principals

---
### Final dataframe
---

#### title.basics
    title.basics.tsv.gz - Contains the following information for titles:

        tconst (string) - alphanumeric unique identifier of the title
        titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
        primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
        originalTitle (string) - original title, in the original language
        isAdult (boolean) - 0: non-adult title; 1: adult title
        startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
        endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
        runtimeMinutes – primary runtime of the title, in minutes
        genres (string array) – includes up to three genres associated with the title

In [2]:
# Df
df = pd.read_table(
    os.path.join("Data", "Raw", "title.basics.tsv"), low_memory=False, na_values="\\N"
).assign(
    # Dummy titre original != titre international
    # Supposément, si le film a un titre international, c'est qu'il s'est exporté à l'étranger ?
    # Ou alors utiliser la variable construite à partir de titles.akas
    # + variable "main_genre", premier élément des listes dans la variable "genres"
    changed_title=lambda x: np.where(x["primaryTitle"] != x["originalTitle"], 1, 0),
    main_genre=lambda x: x["genres"].str.split(",", expand=True)[0],
)
print("Nrows raw data:", len(df))

# Merge with film ratings
df = pd.merge(
    df,
    pd.read_table(os.path.join("Data", "Raw", "title.ratings.tsv"), na_values="\\N"),
    how="left",
    on="tconst",
).loc[
    lambda x: x["averageRating"].notna()
]  # drop null average ratings
print("Nrows data after null ratings deletion:", len(df))

# Merge with number of titles
df = pd.merge(
    df,
    pd.read_feather(os.path.join("Data", "Modified", "ntitles.feather")).rename(
        columns={"titleId": "tconst"}
    ),
    how="left",
    on="tconst",
)

# Merge with crew ratings
df = pd.merge(
    df,
    pd.read_feather(os.path.join("Data", "Modified", "crew.ratings.feather")),
    how="left",
    on="tconst",
)

# Merge with maxSeason
df = pd.merge(
    df,
    pd.read_feather(os.path.join("Data", "Modified", "series.seasons.feather")).rename(columns={'parentTconst':'tconst'}),
    how='left',
    on='tconst'
)

# Merge with crew size indice
df = (
    pd.merge(
        df,
        pd.read_feather(os.path.join("Data", "Modified", "crew.size.indice.ftr")),
        how="left",
        on="tconst",
    )
    .assign(
        # sumVotes & average ratings for cast
        sumVotes_crew=lambda x: x["sumVotes_Actors"].fillna(0)
        + x["sumVotes_Director"].fillna(0)
        + x["sumVotes_Others"].fillna(0),
        avgRating_crew=lambda x: (
            x["avgRating_weighted_Actors"].fillna(0) * x["sumVotes_Actors"].fillna(0)
            + x["avgRating_weighted_Director"].fillna(0)
            * x["sumVotes_Director"].fillna(0)
            + x["avgRating_weighted_Others"].fillna(0) * x["sumVotes_Others"].fillna(0)
        )
        / (
            x["sumVotes_Actors"].fillna(0)
            + x["sumVotes_Director"].fillna(0)
            + x["sumVotes_Others"].fillna(0)
        )
    )
)

display(df)


# Delete some types of movies
print("List of types + value counts: \n", df["titleType"].value_counts())

musicvideos = (df["titleType"] == "video") & (df["genres"] == "Music,Short") # Trop différent
# tvseries = (df["titleType"] == "tvSeries") | (df["titleType"] == "tvMiniSeries") # Trop différent aussi (il faudrait le nb de saisons, notamment, ou alors chaque saison individuellement ?)
tvepisodes = df["titleType"] == "tvEpisode" # Pas très intéressant je trouve. Et pas grand-monde note les épisodes individuellement
videogames = df["titleType"] == "videoGame" # Encore plus différent
shorts = (df["titleType"] == "tvShort") | (df["titleType"] == "short") # Trop différent aussi
tvSpecial = (df["titleType"] == "tvSpecial")

print("Deleted types: musicvideos, tvepisodes, videogames, shorts, tvSpecial")
(df.drop([el for el in df.loc[musicvideos | tvepisodes | videogames | shorts | tvSpecial].index], inplace=True))

del musicvideos, tvepisodes, videogames, shorts, tvSpecial

print("Final df size:", len(df))

# Last few variables: mainstream (15% of films with the most votes) + log_numVotes
# Deleted endYear -> not used.

# Mainstream variable threshold
print("Mainstream variable threshold:", df["numVotes"].quantile(q=0.85), "votes")
df = (
    df.assign(
        ## Variable "mainstream/obscur"
        mainstream=lambda x: (
            np.where(x["numVotes"] >= x["numVotes"].quantile(q=0.85), 1, 0)
        ),
        log_numVotes=lambda x: np.log(x["numVotes"]),
    )
    .drop(columns=["endYear"])
    ## those 0 votes should be NaN (it's because I used fillna(0) earlier)
    .replace({"sumVotes_crew": 0}, np.NaN)
    .reset_index(drop=True)
)

df.to_csv(os.path.join("Data", "Modified", "imdb_main_vars.csv"))
print("Saved to", os.path.join("Data", "Modified", "imdb_main_vars.csv"))

Nrows raw data: 8014408
Nrows data after null ratings deletion: 1164725


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,changed_title,...,sumVotes_Actors,sumVotes_Director,sumVotes_Others,avgRating_weighted_Actors,avgRating_weighted_Director,avgRating_weighted_Others,maxSeason,crew_size_indice,sumVotes_crew,avgRating_crew
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short",0,...,1750.0,1988.0,2043.0,5.702171,5.656087,5.322810,NaN,3.0,5781.0,5.552257
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short",0,...,0.0,1566.0,1609.0,NaN,6.415773,6.465071,NaN,2.0,3175.0,6.440756
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4,"Animation,Comedy,Romance",0,...,0.0,1566.0,4573.0,NaN,6.415773,6.488192,NaN,4.0,6139.0,6.469718
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12,"Animation,Short",0,...,0.0,1566.0,1609.0,NaN,6.415773,6.465071,NaN,2.0,3175.0,6.440756
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,"Comedy,Short",0,...,4548.0,1988.0,4558.0,6.200000,5.656087,6.202479,NaN,4.0,11094.0,6.103551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164720,tt9916682,tvEpisode,Horrid Henry It's all Your Fault,Horrid Henry It's all Your Fault,0.0,2012.0,NaN,10,"Animation,Comedy,Family",0,...,2191414.0,45584.0,18253.0,7.445591,6.758178,6.182337,NaN,10.0,2255251.0,7.421473
1164721,tt9916690,tvEpisode,Horrid Henry Delivers the Milk,Horrid Henry Delivers the Milk,0.0,2012.0,NaN,10,"Animation,Comedy,Family",0,...,2191414.0,45584.0,18253.0,7.445591,6.758178,6.182337,NaN,10.0,2255251.0,7.421473
1164722,tt9916720,short,The Nun 2,The Nun 2,0.0,2019.0,NaN,10,"Comedy,Horror,Mystery",0,...,228.0,30.0,1048186.0,5.984649,5.200000,6.968431,NaN,7.0,1048444.0,6.968166
1164723,tt9916766,tvEpisode,Episode #10.15,Episode #10.15,0.0,2019.0,NaN,43,"Family,Reality-TV",0,...,318937.0,0.0,105166.0,6.688077,NaN,6.518123,NaN,10.0,424103.0,6.645933


List of types + value counts: 
 tvEpisode       543404
movie           264961
short           133516
tvSeries         76262
video            63471
tvMovie          47193
videoGame        12170
tvMiniSeries     11819
tvSpecial         9716
tvShort           2213
Name: titleType, dtype: int64
Deleted types: musicvideos, tvepisodes, videogames, shorts, tvSpecial
Final df size: 448997
Mainstream variable threshold: 462.0 votes
Saved to Data\Modified\imdb_main_vars.csv


#### Get movies descriptions from OMDb

At the time of writing, this df has a around 450K observations. If somehow you launch this in the future, the number of observations is likely to be (a lot?) bigger, so you will have to adjust the code accordingly. I admit this could be automated better, but I did not want to spend too much time on this.

In [8]:
df = df.reset_index().loc[:,["tconst", "primaryTitle"]]

df_day1_1 = df.iloc[0:50000]
df_day1_2 = df.iloc[50000:100000]
df_day2_1 = df.iloc[100000:150000]
df_day2_2 = df.iloc[150000:200000]
df_day3_1 = df.iloc[200000:250000]
df_day3_2 = df.iloc[250000:300000]
df_day4_1 = df.iloc[300000:350000]
df_day4_2 = df.iloc[350000:400000]
df_day5_1 = df.iloc[400000:450000]

# del df

In [47]:
# * Launched 26/06/2021, 15:47
df_day1_1 = df_day1_1.apply(getOMDbMovieData,axis=1, **{'filename_csv':"day1_1.csv"})

In [79]:
# * Launched 26/06/2021, 21:10 
# ! JSON error with tconst tt0120690
# ! I have no idea what to do. I'll skip this one.
df_day1_2 = df_day1_2.drop(df_day1_2.loc[df_day1_2["tconst"]=="tt0120690"].index).apply(getOMDbMovieData,axis=1, **{'filename_csv':"day1_2.csv"})

In [9]:
# * Launched 28/06/2021, 10:31
df_day2_1 = df_day2_1.apply(getOMDbMovieData, axis=1, **{'filename_csv':"day2_1.csv"})

In [8]:
# * Launched 27/06/2021, 09:02
# ! removed tt0420900 because it's a visual novel
# (also because it's Evangelion; I bought the box set when I was 14 since a friend couldn't stop talking about it, but I found it boring.
# I still feel betrayed to this day. I want my 20 bucks back. https://www.amazon.fr/Evangelion-Neon-Genesis-Int%C3%A9grale-Platinum/dp/B002C3JVHG/)
df_day2_2 = df_day2_2.drop(df_day2_2.loc[df_day2_2["tconst"]=="tt0420900"].index).apply(getOMDbMovieData, axis=1, **{'filename_csv':"day2_2.csv"})

In [9]:
# * Launched 28/06/2021, 10:31
# ! tt0815143 error
df_day3_1 = df_day3_1.drop(df_day3_1.loc[df_day3_1["tconst"]=="tt0815143"].index).apply(getOMDbMovieData, axis=1, **{'filename_csv':"day3_1.csv"})

In [9]:
# * Launched 30/06/2021, 19:42
# ! tt1350496 error
df_day3_2 = df_day3_2.drop(df_day3_2.loc[df_day3_2["tconst"]=="tt1350496"].index).apply(getOMDbMovieData, axis=1, **{'filename_csv':"day3_2.csv"})

In [7]:
# * Launched 01/07/2021, 19:25
# ! tt1928243 tt2091413 tt2091423 error
df_day4_1 = df_day4_1.drop(df_day4_1.loc[df_day4_1["tconst"].isin(["tt1928243", "tt2091413", "tt2091423"])].index).apply(getOMDbMovieData, axis=1, **{'filename_csv':"day4_1.csv"})

In [10]:
# * Launched 04/07/2021
# ! tt4601148 error. 
# I actually wonder why I get this error. Sometimes it seems to works if I wait for a bit?
df_day4_2 = df_day4_2.drop(df_day4_2.loc[df_day4_2["tconst"]=="tt4601148"].index).apply(getOMDbMovieData, axis=1, **{'filename_csv':"day4_2.csv"})

In [11]:
# * Launched 04/07/2021
df_day5_1 = df_day5_1.drop(df_day5_1.loc[df_day5_1["tconst"].isin(["tt5967998", "tt6959402", "tt6984724"])].index).apply(getOMDbMovieData, axis=1, **{'filename_csv':"day5_1.csv"})

In [12]:
# Merging OMDb data with imdb_main_vars to create final dataframe
df = (
    pd.read_csv(os.path.join("Data", "Modified", "imdb_main_vars.csv"))
    # Get descriptions
    .merge(
        right=pd.concat(
            [
                pd.read_csv(os.path.join("Data", "Modified", file))
                .drop_duplicates()
                .drop(
                    columns=["primaryTitle", "api_title"]
                )  # There are a few duplicates here and there. Oops.
                for file in os.listdir(os.path.join("Data", "Modified"))
                if file.startswith("day")
            ],
            axis=0,
        ).assign(
            # Cleaning language variable (deleting a lot of "None")
            language=lambda x: x["language"]
            .str.replace("None, |, None", "", regex=True)
            .replace("None", np.NaN)
        ),
        on="tconst",
        how="left",
    )
    # Get names of crew
    .merge(
        pd.read_feather(os.path.join("Data", "Modified", "crew.names.feather")),
        on="tconst",
        how="left",
    )
    # Names = Vector with all names
    # awards_nom_or_won = binary. 1 if the field was nominated for an award, or if it has won one.
    # awards_won = binary. 1 if the film has won an award
    # all_texts = dump of the genres, countries, languages, crew names, and plot of the film
    .assign(
        Names=lambda x: x["names_Actors"].fillna("")
        + ", "
        + x["names_Director"].fillna("")
        + ", "
        + x["names_Others"].fillna(""),
        awards_nom_or_won=lambda x: np.where(
            x["awards"].notna(), "Nominated or won awards", "No awards"
        ),
        awards_won=lambda x: np.where(
            x["awards"].fillna("").str.lower().str.contains("win|won", regex=True),
            "Awards won",
            "No awards won",
        ),
        all_texts=lambda x: x["genres"].fillna("").str.replace(",", " ")
        + " "
        + x["country"].fillna("").str.replace(",", " ")
        + " "
        + x["language"].fillna("").str.replace(",", " ")
        + " "
        + x["Names"].fillna("").str.replace(",", " ")
        + " "
        + x["plot"].fillna(""),
    ).drop(columns=["Unnamed: 0"])
)

print("Df length full:", len(df))
print(
    "Df length with descriptions:",
    len(df.dropna(axis=0, how="any", subset=["plot"])),
)
print(
    "Films with descriptions:",
    round((len(df.dropna(axis=0, how="any", subset=["plot"])) / len(df)) * 100, 2),
    "%",
    sep="",
)

# Final df
df.to_csv(os.path.join("Data", "Final", "imdb.csv"))

Df length full: 448997
Df length with descriptions: 312152
Films with descriptions:69.52%
